<a href="https://colab.research.google.com/github/pranay8297/fastaip2/blob/main/ml_coding_interview_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [228]:
import numpy as np

In [244]:
# Lets start with Micro grad

class Value:

    def __init__(self, val, parents = (), op = None, label = None):
        self.val = val
        self.parents = parents
        self.op = op
        self.grad = 0
        self.label = label

    def __repr__(self):
        return f'Value: {self.val:.4f}' + (f'  :: Label: {self.label}' if self.label != None else '')

    def __sub__(self, other):
        return self + (-other)

    def __neg__(self):
        return self * -1

    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * (other**-1)

    def __add__(self, other):

        if not isinstance(other, Value): other = Value(other, label = str(other))

        out = Value(self.val + other.val, (self, other), label = f'{self.label}+{other.label}')

        def _backward():
            self.grad += 1 * out.grad
            other.grad += 1 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        if not isinstance(other, Value): other = Value(other, label = str(other))
        out = Value(self.val * other.val, (self, other), label = f'{self.label}*{other.label}')

        def _backward():
            self.grad += other.val * out.grad
            other.grad += self.val * out.grad

        out._backward = _backward
        return out

    def __pow__(self, pow):
        out = Value(self.val ** pow, (self,), label = f'{self.label}**{pow}')
        def _backward():
            self.grad += pow*self.val**(pow-1) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        exp = np.exp(self.val)
        out = Value(exp, (self,), label = f'exp({self.label})')
        def _backward():
            self.grad += exp * out.grad
        out._backward = _backward
        return out

    def relu(self):
        out = Value(self.val if self.val > 0 else 0, label = f'relu({self.label})')

        def _backward():
            self.grad += 0 if out.val else out.grad

        out._backward = _backward
        return out

    def backward(self):
        self.grad = 1.
        visited = set()
        dfs = []
        # breakpoint()
        def _dfs(node):

            if node in visited: return

            visited.add(node)

            for i in node.parents:
                _dfs(i)

            dfs.append(node)

        _dfs(self)
        topsort = reversed(dfs)

        for i in topsort:
            if not hasattr(i, '_backward'): continue
            i._backward()


In [230]:
x1, x2 = Value(-1.5, label = 'x1'), Value(1.3, label = 'x2')
w1, w2 = Value(4, label = 'w1'), Value(5, label = 'w2')
b = Value(0, label = 'b')
z = w1*x1 + w2*x2 + b
z.label = 'z'

# z.backward()
# w1.grad, w2.grad, b.grad
print(z)
yhat = ((2*z).exp() - 1)/((2*z).exp() + 1)
# yhat.label = 'yhat'
print(yhat)

yhat.backward()

yhat.grad, z.grad, w1.grad, w2.grad, b.grad

Value: 0.5000  :: Label: z
Value: 0.4621  :: Label: exp(z*2)+-1*exp(z*2)+1**-1


(1.0,
 0.7864477329659272,
 -1.1796715994488909,
 1.0223820528557053,
 0.7864477329659272)

In [286]:
import random

class Module:
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

class Neuron(Module):
    def __init__(self, n_in):
        self.ws = [Value(random.uniform(-1, 1)) for _ in range(n_in)]
        self.b = Value(0)

    def forward(self, x): # dot product + b: wx + b
        # breakpoint()
        return sum((a * b for a, b in zip(self.ws, x))) + self.b

    def __repr__(self):
        return f"Neuron containing: {self.ws}"

class Layer(Module):
    def __init__(self, n_in, n_out):
        # n_in: number of inputs
        # n_out: number of outputs desired in a layer
        self.neurons = [Neuron(n_in) for _ in range(n_out)]

    def forward(self, x):
        return [n(x) for n in self.neurons]

    def __repr__(self): return f"Layer containing weights of shape: {len(self.neurons)}x{len(self.neurons[0].ws)}"


class MLP(Module):

    def __init__(self, lc = [], act = 'relu'):
        # assume lc - layer_config has n_in preppended
        self.layers = [Layer(lc[i-1], lc[i]) for i in range(1, len(lc))]

    def forward(self, x):

        for layer in self.layers:
            x = layer(x)
        return x

    def __repr__(self):
        st = ''
        for i in self.layers:
            st += i.__repr__() + ' \n'
        return st

In [288]:
x = [Value(1, label = 'x1'), Value(1.5, label = 'x2'), Value(1.2, label = 'x3')]
neu = Neuron(3)
neu(x)

Value: -1.1044  :: Label: None*x1+0+None*x2+None*x3+None

In [289]:
x = [Value(1, label = 'x1'), Value(1.5, label = 'x2'), Value(1.2, label = 'x3')]
layer = Layer(3, 2)
inter = layer(x)
inter

[Value: 1.2700  :: Label: None*x1+0+None*x2+None*x3+None,
 Value: 1.3024  :: Label: None*x1+0+None*x2+None*x3+None]

In [290]:
x = [Value(1, label = 'x1'), Value(1.5, label = 'x2'), Value(1.2, label = 'x3')]
mlp = MLP((3, 2, 1))
yhat = mlp(x)

In [293]:
yhat[0].backward()
yhat[0].val

-0.13525551427592739

In [294]:
for layer in mlp.layers:
    for neu in layer.neurons:
        print([i.grad for i in neu.ws])

[0.29952624858971144, 0.44928937288456716, 0.3594314983076537]
[-0.5134056657947423, -0.7701084986921135, -0.6160867989536908]
[-0.5655506771842844, -0.06650054863652788]
